In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.callbacks import Callback, CSVLogger
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical
import argparse
import os
import cv2
from PIL import Image
import numpy as np

Using TensorFlow backend.
C:\euler\Anaconda3\envs\py36_django_mnist_git\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\euler\Anaconda3\envs\py36_django_mnist_git\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\euler\Anaconda3\envs\py36_django_mnist_git\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\euler\Anaconda

In [2]:
def conver_to_mnist(convas_jpg):
    # 画像が RGB ならグレースケールに変換する。(28, 28, 3) -> (28, 28)
    img_gray = convas_jpg.convert("L")
    size = (28, 28)
    img_gray = img_gray.resize(size)

    # 行列を1次元に変換する(28, 28) -> (784) にする。
    data = np.array(img_gray).reshape(28, 28, 1)
    mnist_data = 255 - data

    # img_mnist = Image.fromarray(mnist_data.reshape(28, -1))
    # img_mnist.save(os.path.join(img_save_path, '{}_input_minist_gray_2.png'.format(rdint)))

    mnist_data = mnist_data.astype('float32') / 255

    return mnist_data

# https://codeday.me/bug/20190505/1055129.html
def load_data(img_dir):
    # return np.array([cv2.imread(os.path.join(img_dir, img)).flatten() for img in os.listdir(img_dir) if img.endswith(".jpg")])

    list_of_imgs = []
    list_of_results = []
    for file_name in os.listdir(img_dir):
        full_name = os.path.join(img_dir, file_name)
        if not full_name.endswith(".jpg"):
            continue
        img = Image.open(full_name)
        if img is None:
            print("Unable to read image", full_name)
            continue
        
        mnist_data = conver_to_mnist(img) 
        
        list_of_imgs.append(mnist_data)
        list_of_results.append(file_name[0])
        
    x = np.array(list_of_imgs)
    y =  np.array(list_of_results)

    return x, y

In [3]:
train_image="train_image"
(x_train, y_train) = load_data(train_image)
# 正解ラベルをone-hot-encoding
y_train = to_categorical(y_train, 10)

print(x_train.shape)
print(y_train.shape)

test_image="test_image"
(x_test, y_test) = load_data(test_image)
# 正解ラベルをone-hot-encoding
y_test = to_categorical(y_test, 10)

print(x_test.shape)
print(y_test.shape)

(1339, 28, 28, 1)
(1339, 10)
(343, 28, 28, 1)
(343, 10)


In [4]:
# create model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               204928    
__________

In [5]:
# train
model.fit(x_train, y_train,
                    batch_size=128, 
                    epochs=35,
                    verbose=1)

Epoch 1/40
1339/1339 [==============================] - 2s 1ms/step - loss: 2.1747 - acc: 0.2061
Epoch 2/40
1339/1339 [==============================] - 1s 783us/step - loss: 1.8166 - acc: 0.3689
Epoch 3/40
1339/1339 [==============================] - 1s 803us/step - loss: 1.5056 - acc: 0.5086
Epoch 4/40
1339/1339 [==============================] - 1s 793us/step - loss: 1.3255 - acc: 0.5713
Epoch 5/40
1339/1339 [==============================] - 1s 685us/step - loss: 1.1506 - acc: 0.6288
Epoch 6/40
1339/1339 [==============================] - 1s 722us/step - loss: 1.0177 - acc: 0.6572
Epoch 7/40
1339/1339 [==============================] - 1s 721us/step - loss: 0.9351 - acc: 0.6833
Epoch 8/40
1339/1339 [==============================] - 1s 749us/step - loss: 0.8516 - acc: 0.7162
Epoch 9/40
1339/1339 [==============================] - 1s 808us/step - loss: 0.7507 - acc: 0.7453
Epoch 10/40
1339/1339 [==============================] - 1s 817us/step - loss: 0.7398 - acc: 0.7535
Epoch 11/40

### 4.モデルを評価する
score = model.evaluate(x_test, y_test, verbose=0)
print(score)

In [7]:
model.save("eulerlcs_cnn.h5")